In [91]:
import pandas as pd
import numpy as np

In [92]:
dt = pd.read_csv("trafficIDS.csv")

/var/folders/06/55v518js15z45m56yvr298780000gn/T/ipykernel_57908/3152150884.py:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv("trafficIDS.csv")


In [93]:
dt.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [94]:
len(dt)

613104

In [95]:
len(dt.loc[dt['Label']=='Benign'])

544200

In [96]:
len(dt.loc[dt['Label']=='Infilteration'])

68871

In [97]:
benign_x =dt.loc[dt['Label']=='Benign']
benign_x = benign_x.loc[0:2999]

In [98]:
malicious_x = dt.loc[dt['Label']=='Infilteration']
malicious_x = malicious_x.iloc[0:300]

In [99]:
proc_x = pd.concat([benign_x, malicious_x],axis=0)
print(len(proc_x))

3300


In [100]:
labels = []
for i in proc_x["Label"]:
    if i == "Benign":
        labels.append(0)
    else:
        labels.append(1)
        
proc_x['anomaly'] = labels
proc_x = proc_x.drop("Label",axis=1)

In [101]:
proc_x = proc_x.drop("Timestamp",axis=1)

In [102]:
for i in proc_x.columns:
    print(i,proc_x.iloc[0][i],type(proc_x.iloc[0][i]))

Dst Port 443 <class 'int'>
Protocol 6 <class 'int'>
Flow Duration 94658 <class 'int'>
Tot Fwd Pkts 6 <class 'int'>
Tot Bwd Pkts 7 <class 'int'>
TotLen Fwd Pkts 708 <class 'int'>
TotLen Bwd Pkts 3718 <class 'int'>
Fwd Pkt Len Max 387 <class 'int'>
Fwd Pkt Len Min 0 <class 'int'>
Fwd Pkt Len Mean 118.0 <class 'float'>
Fwd Pkt Len Std 159.2846508613 <class 'float'>
Bwd Pkt Len Max 1460 <class 'int'>
Bwd Pkt Len Min 0 <class 'int'>
Bwd Pkt Len Mean 531.1428571429 <class 'float'>
Bwd Pkt Len Std 673.1182235367 <class 'float'>
Flow Byts/s 46757.8017705846 <class 'float'>
Flow Pkts/s 137.3365167234 <class 'float'>
Flow IAT Mean 7888.1666666667 <class 'float'>
Flow IAT Std 11130.0425943262 <class 'float'>
Flow IAT Max 24325 <class 'int'>
Flow IAT Min 0 <class 'int'>
Fwd IAT Tot 72880 <class 'int'>
Fwd IAT Mean 14576.0 <class 'float'>
Fwd IAT Std 12590.3839695221 <class 'float'>
Fwd IAT Max 24385 <class 'int'>
Fwd IAT Min 363 <class 'int'>
Bwd IAT Tot 72178 <class 'int'>
Bwd IAT Mean 12029.6666

In [104]:
for i in proc_x.columns:
    proc_x[i] = pd.to_numeric(proc_x[i])

In [105]:
# Replace to drop rows or columns infinite values
proc_x = proc_x.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
print(proc_x)

        Dst Port  Protocol  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  \
index                                                                   
0            443         6          94658             6             7   
1            443         6            206             2             0   
2            445         6         165505             3             1   
3            443         6         102429             6             7   
4            443         6            167             2             0   
...          ...       ...            ...           ...           ...   
382197       443         6        4589218            11            12   
382198       443         6            274             3             0   
382199       445         6         628530             3             1   
382200       443         6       11746166             6             8   
382201       443         6            246             3             0   

        TotLen Fwd Pkts  TotLen Bwd Pkts  Fwd Pkt 

In [108]:
proc_x = proc_x.reset_index()
proc_x = proc_x.drop("index",axis=1)
proc_x.index.name = "index"

In [115]:
proc_x.columns = [col.replace(" ","_").lower() for col in proc_x.columns]
proc_x.columns

Index(['dst_port', 'protocol', 'flow_duration', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'flow_byts/s', 'flow_pkts/s', 'flow_iat_mean',
       'flow_iat_std', 'flow_iat_max', 'flow_iat_min', 'fwd_iat_tot',
       'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max', 'fwd_iat_min',
       'bwd_iat_tot', 'bwd_iat_mean', 'bwd_iat_std', 'bwd_iat_max',
       'bwd_iat_min', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fwd_header_len', 'bwd_header_len', 'fwd_pkts/s',
       'bwd_pkts/s', 'pkt_len_min', 'pkt_len_max', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fin_flag_cnt', 'syn_flag_cnt',
       'rst_flag_cnt', 'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt',
       'cwe_flag_count', 'ece_flag_cnt', 'down/up_ratio', 'pkt_size_avg',
       'fwd_seg_siz

In [116]:
proc_x.to_csv("input_data.txt")